# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather = pd.read_csv("../output_data/cities.csv")
weather


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
...,...,...,...,...,...,...,...,...,...,...
543,543,yabrud,0,SY,1558378840,32,33.97,36.66,80.60,8.05
544,544,paraiso,5,MX,1558378898,4,24.01,-104.61,84.20,16.11
545,545,veraval,0,FR,1558378898,62,49.65,0.71,61.00,8.05
546,546,novyy urgal,100,RU,1558378899,93,51.07,132.56,49.74,2.68


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = weather["Humidity"]

In [4]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=np.max(humidity), point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 60 and 80
narrowedcity= weather.loc[(weather["Wind Speed"] <= 10) & (weather["Cloudiness"] == 0) & \
                                   (weather["Max Temp"] >= 60) & (weather["Max Temp"] <= 80)].dropna()

narrowedcity

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
37,37,varhaug,0,NO,1558378761,63,58.61,5.65,64.99,8.05
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
84,84,emerald,0,AU,1558378770,82,-23.53,148.16,60.80,6.93
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
141,141,sanchazi,0,CN,1558378781,82,30.43,103.62,64.40,2.24
168,168,straumen,0,NO,1558378786,52,63.87,11.30,68.00,9.17
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93
192,192,port hedland,0,AU,1558378791,51,-20.31,118.58,62.60,6.93


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create a hotel_df
hotel = narrowedcity.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel["Hotel Name"] = ""

# Display the result
hotel

,City,Country,Lat,Lng,Hotel Name
37,varhaug,NO,58.61,5.65,
70,ponta do sol,BR,-20.63,-46.00,
84,emerald,AU,-23.53,148.16,
88,cidreira,BR,-30.17,-50.22,
138,morondava,MG,-20.30,44.28,
139,vaini,IN,15.34,74.49,
141,sanchazi,CN,30.43,103.62,
168,straumen,NO,63.87,11.30,
176,nador,MA,35.17,-2.93,
192,port hedland,AU,-20.31,118.58,


In [7]:

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}


for index, row in hotel.iterrows():
    #variables name (lng, lat, city from the dataframe)
    lng = row["Lng"]
    lat = row["Lat"]
    city = row["City"]
    
    params["location"] = f"{lat},{lng}"

    # API request
    print(f"Results for  {index}: {city}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    # save hotel name to dataframe
    try:
        print(f"Closest hotel in {city} is {results[0]['name']}.")
        hotel.loc[index, "Hotel Name"] = results[0]['name']

    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    


# Print end of search once searching is completed
print("-------End of Search-------")




Results for  37: varhaug.
Closest hotel in varhaug is Huset ved havet.
------------
Results for  70: ponta do sol.
Closest hotel in ponta do sol is Kanto Island Inn Suites & Spa.
------------
Results for  84: emerald.
Closest hotel in emerald is Emerald Maraboon Motor Inn.
------------
Results for  88: cidreira.
Closest hotel in cidreira is Pousada Vila Di Italy.
------------
Results for  138: morondava.
Closest hotel in morondava is Palissandre Cote Ouest resort & SPA.
------------
Results for  139: vaini.
Closest hotel in vaini is Dandeli Lake County.
------------
Results for  141: sanchazi.
Missing field/result... skipping.
------------
Results for  168: straumen.
Closest hotel in straumen is Jægtvolden Fjordhotell.
------------
Results for  176: nador.
Closest hotel in nador is Hotel Marchica Lagoon Resort, Nador Morocco.
------------
Results for  192: port hedland.
Closest hotel in port hedland is The Esplanade Hotel.
------------
Results for  214: mount isa.
Closest hotel in moun

In [8]:
hotel.head()

,City,Country,Lat,Lng,Hotel Name
37,varhaug,NO,58.61,5.65,Huset ved havet
70,ponta do sol,BR,-20.63,-46.00,Kanto Island Inn Suites & Spa
84,emerald,AU,-23.53,148.16,Emerald Maraboon Motor Inn
88,cidreira,BR,-30.17,-50.22,Pousada Vila Di Italy
138,morondava,MG,-20.30,44.28,Palissandre Cote Ouest resort & SPA


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
#hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
hotel_info = [info_box_template.format(**row) for index, row in hotel.iterrows()]
locations = hotel[["Lat", "Lng"]]

In [10]:
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig


Figure(layout=FigureLayout(height='420px'))